# **Step 1: Business Problem Understanding**
- Based on given car features, predict the **selling price of used car.**
- Required R2 is 0.85 on both train and test data.

In [334]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# **Step 2 : Data Understanding**

In [ ]:
df = pd.read_excel("cars_data.xlsx")
df.head()

In [ ]:
df.columns


**Independent variables**
 - **Name:** Car Model name along with purchase year of the car.
 - **Rating:** Rating given while car inspection by cars 24 team.
 - **City:** City of that car placed for advertainment, city is given interms of code format.
 - **Kilometres:** How many kilometres that particular car has driven for before placing that advertisement.
 - **Year of purchase:** Original Date of purchase of that car.
 - **Owner:** How many previous owners it had before selling on cars24 site.
 - **Fuel type:** Type of fuel which car runs on. (Petrol, diesel...)
 - **Transmission:** whether car is automated or manual transmission.
 - **RTO:** Car registered under which RTO
 - **Insurance:** expiry date of the insurance if any
 - **Insurance Type:** Different t types insurance availed by owner.


**Target Variable**
 - **Price:** Price of the used car

In [ ]:
df.info()

In [ ]:
df['Name'].nunique()

In [ ]:
df['RTO'].nunique()

In [ ]:
df['Insurance'].nunique()

Drop Unimportant columns as per the Feature selection (**Filter Methods**)

In [341]:
df.drop(columns=['Name','RTO','Insurance'],inplace=True)

In [ ]:
df['Rating'].unique()

In [ ]:
df['Rating'].value_counts()

In [ ]:
print(df['city'].unique())
print(df['city'].value_counts())


In [ ]:
print(df['Fuel_Type'].unique())
print(df['Fuel_Type'].value_counts())

In [ ]:
print(df['Insurance_Type'].unique())
print(df['Insurance_Type'].value_counts())

**Treating wrong data type**

In [347]:
# Remove the '₹' symbol and commas, then convert to numeric
df['Price'] = df['Price'].str.replace('₹', '', regex=False).str.replace(',', '', regex=False).astype(float)


In [348]:
# Remove 'out of 5' and extract the numeric rating
df['Rating'] = df['Rating'].str.replace('out of 5', '', regex=False).astype(float)


In [349]:
# Remove 'Kilometers' and 'km', then convert to numeric
df['Kilometers'] = df['Kilometers'].str.replace('Kilometers', '', regex=False).str.replace('km', '', regex=False).str.replace(',', '').astype(float)


In [350]:
# Extract the year from 'Year_of_Purchase' and convert to numeric
df['Year_of_Purchase'] = df['Year_of_Purchase'].str.extract('(\d{4})').astype(int)


In [351]:
# Extract ownership type (e.g., First Owner, Second Owner)
df['Owner'] = df['Owner'].str.extract('(First|Second|Third|Fourth|Fifth) Owner')


In [352]:
# Remove extra text from 'Transmission' column
df['Transmission'] = df['Transmission'].str.replace('Transmission', '', regex=False)

In [353]:
# Remove extra text from 'Transmission' column
df['Insurance_Type'] = df['Insurance_Type'].str.replace('Insurance Type', '', regex=False)

In [ ]:
df.head()

In [ ]:
df['Fuel_Type'].value_counts()

In [ ]:
df['Price'].describe()

In [ ]:
df['Rating'].value_counts()

In [ ]:
df['Kilometers'].describe()

In [ ]:
df['Owner'].value_counts()

In [ ]:
df['Insurance_Type'].value_counts()

**Exploratoy Data analysis (EDA)**

In [ ]:
df.columns

In [362]:
continous = ['Price','Kilometers']

discrete_categorical = ['Owner','Fuel_Type', 'Transmission', 'Insurance_Type']

discrete_count =['Year_of_Purchase','Rating','city']

**For continous Varible**

In [ ]:
plt.rcParams['figure.figsize'] = (18,8)

plt.subplot(1,2,1)
sns.histplot(df['Price'],kde=True)

plt.subplot(1,2,2)
sns.histplot(df['Kilometers'],kde=True)

plt.suptitle('Univariate Analysis on Numerical Columns')
plt.show()

In [ ]:
df[continous].skew()

In [ ]:
sns.heatmap(df[continous].corr(), annot=True)
plt.show()

In [ ]:
sns.pairplot(df[continous])
plt.show()

**For discrete Varible**

In [ ]:
df[discrete_categorical].describe().T

In [ ]:
plt.rcParams['figure.figsize'] = (18,8)

plt.subplot(2,2,1)
sns.countplot(df['Owner'])

plt.subplot(2,2,2)
sns.countplot(df['Fuel_Type'])

plt.subplot(2,2,3)
sns.countplot(df['Transmission'])

plt.subplot(2,2,4)
sns.countplot(df['Insurance_Type'])

plt.suptitle('Univariate Analysis on Categorical Columns')
plt.show()

**Checking the Missing values**

In [ ]:
df.isnull().sum()

In [370]:
# Fill missing values in 'Transmission' with the mode
transmission_mode = df['Transmission'].mode()[0]
df['Transmission'].fillna(transmission_mode, inplace=True)


In [371]:
# Fill missing values in 'owner' with the mode
owner_mode = df['Owner'].mode()[0]
df['Owner'].fillna(owner_mode,inplace=True)

**Checking the Outliers**

In [ ]:
# Visulize using boxplot

plt.subplot(1,2,1)
sns.boxplot(df['Price'])

plt.subplot(1,2,2)
sns.boxplot(df['Kilometers'])

plt.suptitle('Outliers in the Data')
plt.show()

# **Step-3 Data Preprocessing**

**Creating new columns**

In [373]:
df['Car_Age'] = 2024 - df['Year_of_Purchase']
df.drop(columns=['Year_of_Purchase'],inplace=True)

In [ ]:
df.head()

In [375]:
# Create 'Is_Insurance_Valid' column: 1 if insurance is valid, 0 otherwise
valid_insurance = ['Comprehensive', 'Third Party', 'Zero Depreciation']
df['Is_Insurance_Valid'] = df['Insurance_Type'].apply(lambda x: 1 if x in valid_insurance else 0)

df.drop(columns=['Insurance_Type'],inplace=True)

In [376]:
# Price per Kilometer (Cost efficiency)
df['Price_per_Kilometer'] = df['Price'] / df['Kilometers']

# Age group: Categorize car age
bins = [0, 5, 10, 15, 20]
labels = ['0-5 years', '6-10 years', '11-15 years', '16+ years']
df['Car_Age_Group'] = pd.cut(df['Car_Age'], bins=bins, labels=labels)

# Insurance status and age interaction
df['Insurance_and_Age'] = df['Is_Insurance_Valid'] * df['Car_Age']

# Normalized price (if scaling wasn't applied globally)
df['Normalized_Price'] = df['Price'] / df['Price'].max()


**Mapping**

In [ ]:
insurance_mapping = {
    'Comprehensive': 'Comprehensive',
    'Comp': 'Comprehensive',
    'Insurance Expired': 'Expired',
    'Expired': 'Expired',
    '3rd Party': 'Third Party',
    'Third_party': 'Third Party',
    'Zero Depreciation': 'Zero Depreciation',
    'Zero_Dep': 'Zero Depreciation',
    'Not Available': 'Unknown',
    'Normal': 'Unknown',
    'NA': 'Unknown',
    'null': 'Unknown'
}

# Apply the mapping to the column
df['Insurance_Type'] = df['Insurance_Type'].replace(insurance_mapping)

# Display the updated value counts
insurance_counts = df['Insurance_Type'].value_counts()
print(insurance_counts)


In [268]:
# Example mapping of city codes to city names
city_mapping = {
    3686: 'City_A',
    5732: 'City_B',
    4709: 'City_C',
    777: 'City_D',
    2423: 'City_E',
    2378: 'City_F',
    290: 'City_G',
    1692: 'City_H',
    2130: 'City_I',
    769: 'City_J'
}

# Replace the city codes with their corresponding city names
df['city'] = df['city'].map(city_mapping).fillna('Unknown')


In [ ]:
df.head()

**Outlier Treatment**

In [270]:
# outlier should be retrained

**Transformation**

In [271]:
from scipy.stats import boxcox
df['Price'],a = boxcox(df['Price'])
df['Kilometers'],c = boxcox(df['Kilometers'])

In [ ]:
df[continous].skew()

**Encoding**

In [274]:
df['city']= df['city'].replace({'City_A':0, 'City_B':1, 'City_C':2,
                                'City_D':3, 'City_E':4, 'City_F':5,
                                'City_G':6, 'City_H':7, 'City_I':8, 'City_J':9})

In [379]:
# Encode Car_Age_Group (if categorical)
df = pd.get_dummies(df, columns=['Car_Age_Group'], drop_first=True)

# Other categorical encodings
df['Owner'] = df['Owner'].astype('category').cat.codes
df['Fuel_Type'] = df['Fuel_Type'].astype('category').cat.codes
df['Transmission'] = df['Transmission'].astype('category').cat.codes


In [ ]:
df.head()

**Feature Scaling**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Select the columns to scale (numerical features)
columns_to_scale = ['Price', 'Kilometers', 'Car_Age']

# Apply MinMaxScaler
scaled_values = scaler.fit_transform(df[columns_to_scale])

# Replace the original columns with scaled values
df[columns_to_scale] = scaled_values

# Display the scaled dataset
df.head()


In [ ]:
df['Insurance_and_Age'].unique()

In [ ]:
df.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()


# Feature Selection

- **Rating:** Higher ratings could reflect better quality or demand.
- **City:** Regional demand and market conditions can influence car prices.
- **Kilometers:** The mileage of the car directly affects its value.
- **Owner:** The number of previous owners affects the perceived reliability and price.
- **Fuel Type:** Different fuel types can affect pricing due to factors like fuel efficiency and environmental concerns.
- **Transmission:** Manual vs automatic transmissions can influence the resale value.
- **Car Age:** Older cars generally have lower prices due to depreciation.
- **Is Insurance Valid:** Valid insurance is often associated with higher car prices.
- **Price per Kilometer:** Derived feature to represent the relationship between car price and mileage.
- **Insurance and Age Interaction:** Combined feature to capture the effect of insurance type on the car’s price relative to its age.

In [394]:
final_features = df[['Normalized_Price',
    'Rating', 'city', 'Kilometers', 'Owner', 'Fuel_Type',
    'Transmission', 'Car_Age', 'Is_Insurance_Valid', 'Price_per_Kilometer',
    'Insurance_and_Age'
]]


In [ ]:
final_features.to_csv('Final.csv',index=False)